In [129]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [185]:
AS= pd.read_csv('../data1/Alaska_2018_19.csv',skiprows=7)

In [174]:
df= pd.DataFrame(AS)
print(len(df))

7490


In [176]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
33,AS,01/03/2018,18.0,N419AS,SEA,06:00,00:00
34,AS,01/03/2018,480.0,N251AK,PDX,05:25,00:00
50,AS,01/04/2018,18.0,N459AS,SEA,06:00,00:00
51,AS,01/04/2018,480.0,N273AK,PDX,05:25,00:00
95,AS,01/06/2019,1406.0,N634VA,LAX,19:12,00:00
...,...,...,...,...,...,...,...
6678,AS,11/03/2018,1406.0,N631VA,LAX,19:11,00:00
6826,AS,11/14/2018,1406.0,N632VA,LAX,19:12,00:00
6840,AS,11/15/2018,1406.0,N362VA,LAX,19:12,00:00
6995,AS,11/26/2018,1420.0,N921VA,LAX,07:58,00:00


In [133]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AS,01/01/2018,18.0,N282AK,SEA,06:00,06:01
1,AS,01/01/2018,480.0,N557AS,PDX,05:25,05:30
2,AS,01/01/2019,18.0,N273AK,SEA,06:04,05:55
3,AS,01/01/2019,58.0,N492AS,PSP,22:30,22:13
4,AS,01/01/2019,102.0,N267AK,SFO,17:25,17:22


In [134]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [135]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AS,01/01/2018,18.0,N282AK,SEA,06:00,06:01
1,AS,01/01/2018,480.0,N557AS,PDX,05:25,05:30
2,AS,01/01/2019,18.0,N273AK,SEA,06:04,05:55
3,AS,01/01/2019,58.0,N492AS,PSP,22:30,22:13
4,AS,01/01/2019,102.0,N267AK,SFO,17:25,17:22


In [136]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01
1,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30
2,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55
3,AS,2019-01-01,58.0,N492AS,PSP,22:30,22:13
4,AS,2019-01-01,102.0,N267AK,SFO,17:25,17:22
...,...,...,...,...,...,...,...
7484,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11
7485,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58
7486,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31
7487,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15


In [137]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01
1,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01
2,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01
3,AS,2019-01-01,58.0,N492AS,PSP,22:30,22:13,2019-01-01
4,AS,2019-01-01,102.0,N267AK,SFO,17:25,17:22,2019-01-01
...,...,...,...,...,...,...,...,...
7484,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31
7485,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31
7486,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31
7487,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31


In [138]:
df.loc[df['Actual Arrival Time']=='24:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [139]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
1707,AS,2018-04-10,1416.0,N835VA,LAX,00:23,24:00,2018-04-11
2150,AS,2019-04-26,104.0,N264AK,SFO,23:55,24:00,2019-04-27
2760,AS,2019-05-18,104.0,N287AK,SFO,23:55,24:00,2019-05-19
3843,AS,2018-06-26,1416.0,N639VA,LAX,00:37,24:00,2018-06-27
4945,AS,2019-08-03,468.0,N453AS,LAX,23:55,24:00,2019-08-04
7220,AS,2018-12-13,1026.0,N625VA,SFO,23:44,24:00,2018-12-14


In [140]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [141]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
33,AS,2018-01-03,18.0,N419AS,SEA,06:00,00:00,2018-01-03
34,AS,2018-01-03,480.0,N251AK,PDX,05:25,00:00,2018-01-03
50,AS,2018-01-04,18.0,N459AS,SEA,06:00,00:00,2018-01-04
51,AS,2018-01-04,480.0,N273AK,PDX,05:25,00:00,2018-01-04
95,AS,2019-01-06,1406.0,N634VA,LAX,19:12,00:00,2019-01-06
...,...,...,...,...,...,...,...,...
6826,AS,2018-11-14,1406.0,N632VA,LAX,19:12,00:00,2018-11-14
6840,AS,2018-11-15,1406.0,N362VA,LAX,19:12,00:00,2018-11-15
6995,AS,2018-11-26,1420.0,N921VA,LAX,07:58,00:00,2018-11-26
7043,AS,2018-11-30,1024.0,N925VA,SFO,22:23,00:00,2018-11-30


In [142]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [143]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [144]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [145]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01,2018-01-01 06:01,2018-01-01 06:00
1,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01,2018-01-01 05:30,2018-01-01 05:25
2,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01,2019-01-01 05:55,2019-01-01 06:04
3,AS,2019-01-01,58.0,N492AS,PSP,22:30,22:13,2019-01-01,2019-01-01 22:13,2019-01-01 22:30
4,AS,2019-01-01,102.0,N267AK,SFO,17:25,17:22,2019-01-01,2019-01-01 17:22,2019-01-01 17:25
...,...,...,...,...,...,...,...,...,...,...
7484,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31,2018-12-31 15:11,2018-12-31 15:30
7485,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31,2018-12-31 18:58,2018-12-31 19:12
7486,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31,2018-12-31 21:31,2018-12-31 22:12
7487,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31,2018-12-31 23:15,2018-12-31 23:57


In [146]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01,2018-01-01 06:01:00,2018-01-01 06:00:00
1,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01,2018-01-01 05:30:00,2018-01-01 05:25:00
2,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01,2019-01-01 05:55:00,2019-01-01 06:04:00
3,AS,2019-01-01,58.0,N492AS,PSP,22:30,22:13,2019-01-01,2019-01-01 22:13:00,2019-01-01 22:30:00
4,AS,2019-01-01,102.0,N267AK,SFO,17:25,17:22,2019-01-01,2019-01-01 17:22:00,2019-01-01 17:25:00
...,...,...,...,...,...,...,...,...,...,...
7484,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31,2018-12-31 15:11:00,2018-12-31 15:30:00
7485,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31,2018-12-31 18:58:00,2018-12-31 19:12:00
7486,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31,2018-12-31 21:31:00,2018-12-31 22:12:00
7487,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31,2018-12-31 23:15:00,2018-12-31 23:57:00


In [147]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01,2018-01-01 06:01:00,2018-01-01 06:00:00,-1.0
1,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01,2018-01-01 05:30:00,2018-01-01 05:25:00,-5.0
2,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01,2019-01-01 05:55:00,2019-01-01 06:04:00,9.0
3,AS,2019-01-01,58.0,N492AS,PSP,22:30,22:13,2019-01-01,2019-01-01 22:13:00,2019-01-01 22:30:00,17.0
4,AS,2019-01-01,102.0,N267AK,SFO,17:25,17:22,2019-01-01,2019-01-01 17:22:00,2019-01-01 17:25:00,3.0
...,...,...,...,...,...,...,...,...,...,...,...
7484,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31,2018-12-31 15:11:00,2018-12-31 15:30:00,19.0
7485,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31,2018-12-31 18:58:00,2018-12-31 19:12:00,14.0
7486,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31,2018-12-31 21:31:00,2018-12-31 22:12:00,41.0
7487,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31,2018-12-31 23:15:00,2018-12-31 23:57:00,42.0


In [165]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early
len(df[df['Difference'] > 0])


4666

In [148]:
type(df['Difference'][0])

numpy.float64